In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import re
import time

def extract_policy_blocks(text, lines_after=1):
    policy_keywords = [
        "사업명", "사업개요", "사업목적", "지원대상", "대상자", "신청대상", "자격요건",
        "신청방법", "지원내용", "지원규모", "융자조건", "대출조건", "금리", "접수기간", "문의처"
    ]

    lines = text.splitlines()
    result = []
    i = 0

    while i < len(lines):
        line = lines[i].strip()
        if any(keyword in line for keyword in policy_keywords):
            block = [line]
            # 뒤에 설명줄도 같이 포함
            for j in range(1, lines_after + 1):
                if i + j < len(lines):
                    block.append(lines[i + j].strip())
            result.append("\n".join(block))
            i += lines_after  # 다음 줄로 건너뜀
        i += 1

    return "\n\n".join(result)

# 크롬 드라이버 설정
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # 브라우저 안 띄우고 실행
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# URL 접속
url = "https://housing.seoul.go.kr/site/main/content/sh01_070400/"
driver.get(url)
time.sleep(2)  # 페이지 로딩 대기

# 렌더링 완료된 HTML 가져오기
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# 주요 콘텐츠 추출
main = soup.find("div", class_="sh-view") or soup.find("div", class_="content")
text = main.get_text(separator='\n') if main else soup.get_text()
text = re.sub(r"\s+", "\n", text).strip()


policy_only = extract_policy_blocks(text, lines_after=10)

with open("정책요약.txt", "w", encoding="utf-8") as f:
    f.write(policy_only)

In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import os
import re

# URL 리스트
urls = [
    "https://housing.seoul.go.kr/site/main/content/sh01_070400/",
    "https://housing.seoul.go.kr/site/main/content/sh01_060513",
    "https://housing.seoul.go.kr/site/main/content/sh01_040901",
    "https://housing.seoul.go.kr/site/main/content/sh01_060508",
    "https://housing.seoul.go.kr/site/main/content/sh01_060503",
    "https://housing.seoul.go.kr/site/main/content/sh01_060506",
    "https://www.lh.or.kr/menu.es?mid=a10401010000",
    "https://www.lh.or.kr/menu.es?mid=a10401020100",
    "https://www.lh.or.kr/menu.es?mid=a10401020200"
]

# 브라우저 옵션 설정 (headless)
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# 저장 폴더 생성
save_folder = "표_스크래핑_txt"
os.makedirs(save_folder, exist_ok=True)

# 크롤링 및 저장
for idx, url in enumerate(urls):
    driver.get(url)
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    tables = soup.find_all('table')

    result_lines = []
    for table in tables:
        for row in table.find_all('tr'):
            cols = row.find_all(['th', 'td'])
            line = " | ".join(col.get_text(strip=True) for col in cols)
            if line:
                result_lines.append(line)
        result_lines.append("-" * 60)

    # 파일 이름 정리 (URL에서 이름 따기)
    name_hint = re.sub(r"[^a-zA-Z0-9가-힣]", "_", url.split("/")[-1] or url.split("=")[-1])
    filename = f"{name_hint}.txt"
    filepath = os.path.join(save_folder, filename)

    # 저장
    with open(filepath, "w", encoding="utf-8") as f:
        f.write(f"[출처 URL] {url}\n\n")
        f.write("\n".join(result_lines))

    print(f"✅ 저장 완료: {filepath}")

driver.quit()


✅ 저장 완료: 표_스크래핑_txt\https___housing_seoul_go_kr_site_main_content_sh01_070400_.txt
✅ 저장 완료: 표_스크래핑_txt\sh01_060513.txt
✅ 저장 완료: 표_스크래핑_txt\sh01_040901.txt
✅ 저장 완료: 표_스크래핑_txt\sh01_060508.txt
✅ 저장 완료: 표_스크래핑_txt\sh01_060503.txt
✅ 저장 완료: 표_스크래핑_txt\sh01_060506.txt
✅ 저장 완료: 표_스크래핑_txt\menu_es_mid_a10401010000.txt
✅ 저장 완료: 표_스크래핑_txt\menu_es_mid_a10401020100.txt
✅ 저장 완료: 표_스크래핑_txt\menu_es_mid_a10401020200.txt


In [2]:
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
import os
load_dotenv()

# 1. txt 파일 리스트
txt_folder = "./표_스크래핑_txt"  # 여기에 txt 파일들 넣어둬
file_paths = [os.path.join(txt_folder, f) for f in os.listdir(txt_folder) if f.endswith(".txt")]

# 2. 문서 로딩
docs = []
for path in file_paths:
    loader = TextLoader(path, encoding="utf-8")
    docs.extend(loader.load())

# 3. 텍스트 분할
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100
)
split_docs = text_splitter.split_documents(docs)

# 4. HuggingFace 임베딩 모델
embeddings = OpenAIEmbeddings(model = 'text-embedding-3-large')

# 5. Chroma DB로 벡터화 및 저장
chroma_path = "./chroma_policy_db"
vectorstore = Chroma.from_documents(
    documents=split_docs,
    embedding=embeddings,
    persist_directory=chroma_path
)


print("✅ ChromaDB에 저장 완료!")


✅ ChromaDB에 저장 완료!


In [1]:
import requests
from bs4 import BeautifulSoup
import os

# ▶ 저장 폴더 생성
os.makedirs("output", exist_ok=True)

# ▶ 스크래핑 대상 URL 목록
urls = [
    "https://youth.seoul.go.kr/infoData/plcyInfo/view.do?plcyBizId=20250316005400210633",
    "https://youth.seoul.go.kr/infoData/plcyInfo/view.do?plcyBizId=20250316005400210632",
    "https://youth.seoul.go.kr/infoData/plcyInfo/view.do?plcyBizId=20250316005400210630",
    "https://youth.seoul.go.kr/infoData/plcyInfo/view.do?plcyBizId=20250316005400210628",
    "https://youth.seoul.go.kr/infoData/plcyInfo/view.do?plcyBizId=20250316005400210627",
    "https://youth.seoul.go.kr/infoData/plcyInfo/view.do?plcyBizId=20250316005400210626",
    "https://youth.seoul.go.kr/infoData/plcyInfo/view.do?plcyBizId=R2024031920824",
    "https://youth.seoul.go.kr/infoData/plcyInfo/view.do?plcyBizId=R2024032020863",
    "https://youth.seoul.go.kr/infoData/plcyInfo/view.do?plcyBizId=R2024041221755",
    "https://youth.seoul.go.kr/infoData/plcyInfo/view.do?plcyBizId=R2024051022688",
    "https://youth.seoul.go.kr/infoData/sprtInfo/view.do?sprtInfoId=62335",
    "https://youth.seoul.go.kr/infoData/sprtInfo/view.do?sprtInfoId=62275",
    "https://youth.seoul.go.kr/infoData/sprtInfo/view.do?sprtInfoId=62127"
]

def extract_text_and_tables(url):
    res = requests.get(url)
    soup = BeautifulSoup(res.text, "html.parser")

    # 본문 내용 추출
    content_div = soup.select_one("div.board-detail-contents") or soup.select_one("div.cont-view")
    content_text = content_div.get_text(separator="\n", strip=True) if content_div else "본문 없음"

    # 표 추출
    tables = soup.find_all("table")
    table_texts = []
    for t_idx, table in enumerate(tables, start=1):
        table_lines = [f"\n[표 {t_idx}]\n"]
        for row in table.find_all("tr"):
            cols = [col.get_text(strip=True) for col in row.find_all(["th", "td"])]
            if cols:
                table_lines.append(" | ".join(cols))
        table_texts.append("\n".join(table_lines))

    return content_text, table_texts

# ▶ 스크래핑 및 저장 실행
for idx, url in enumerate(urls, start=1):
    print(f"▶ {idx}번 URL 스크래핑 중...")

    text, table_texts = extract_text_and_tables(url)
    base_filename = f"output/seoul_youth_{idx:02d}.txt"

    with open(base_filename, "w", encoding="utf-8") as f:
        f.write(f"[URL]\n{url}\n\n")
        f.write("[본문]\n")
        f.write(text + "\n\n")
        for table_text in table_texts:
            f.write(table_text + "\n")

    print(f"✔ 저장 완료: {base_filename}")

print("\n✅ 모든 `.txt` 저장 완료!")


▶ 1번 URL 스크래핑 중...
✔ 저장 완료: output/seoul_youth_01.txt
▶ 2번 URL 스크래핑 중...
✔ 저장 완료: output/seoul_youth_02.txt
▶ 3번 URL 스크래핑 중...
✔ 저장 완료: output/seoul_youth_03.txt
▶ 4번 URL 스크래핑 중...
✔ 저장 완료: output/seoul_youth_04.txt
▶ 5번 URL 스크래핑 중...
✔ 저장 완료: output/seoul_youth_05.txt
▶ 6번 URL 스크래핑 중...
✔ 저장 완료: output/seoul_youth_06.txt
▶ 7번 URL 스크래핑 중...
✔ 저장 완료: output/seoul_youth_07.txt
▶ 8번 URL 스크래핑 중...
✔ 저장 완료: output/seoul_youth_08.txt
▶ 9번 URL 스크래핑 중...
✔ 저장 완료: output/seoul_youth_09.txt
▶ 10번 URL 스크래핑 중...
✔ 저장 완료: output/seoul_youth_10.txt
▶ 11번 URL 스크래핑 중...
✔ 저장 완료: output/seoul_youth_11.txt
▶ 12번 URL 스크래핑 중...
✔ 저장 완료: output/seoul_youth_12.txt
▶ 13번 URL 스크래핑 중...
✔ 저장 완료: output/seoul_youth_13.txt

✅ 모든 `.txt` 저장 완료!
